In [1]:
import numpy as np
import pandas as pd
import os
import json
import re
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from opencc import OpenCC

In [2]:
# switch model
model = tf.keras.models.load_model('./Model/LSTM_OpLabV2.h5')

2023-12-09 23:37:19.286688: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-12-09 23:37:19.286712: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-12-09 23:37:19.286716: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-12-09 23:37:19.286746: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-09 23:37:19.286760: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# 文字轉數字(index)
with open('./Model/LSTM_OpLabV2_dic.json', 'r', encoding='utf-8') as fileRead:
        word2Index = json.load(fileRead)
index2Word = {word2Index[word]:word for word in word2Index}

In [25]:
# 預測文字，並把預測文字循環當作下一次的輸入

# 設定你的temperature
temperature = 0.005
seq_len = 100

def generateWords(input,words=5, wordCount=100):
    paragraph = ""
    for ind in input:
        paragraph += index2Word[ind]
        
    while words>0 and len(paragraph) < wordCount:
        next_input = tf.expand_dims(input,axis=0)
        predicts = model(next_input)
        predicts = predicts[:,-1,:]
        predicts /= temperature
        result = tf.random.categorical(
            predicts,num_samples=1
        )
        chinese_ind = tf.squeeze(result).numpy()
        # print(index2Word[chinese_ind],end="")
        paragraph += index2Word[chinese_ind]
        input = input+[chinese_ind]
        input = input[-seq_len:]
        if index2Word[input[-1]] == "。":
            words -=1
    return paragraph

In [26]:
cc = OpenCC('t2s')
init_seq = cc.convert("有鬼靠")
init_seq_ind = [word2Index[w] for w in init_seq]
input = init_seq_ind[-seq_len:]

result = generateWords(input,20,2000)

In [27]:
result

'有鬼靠近。”“那你说说，你们看看我们……”“什么？”“我们是来找你的。”“那你们……”“我们是来找你的，我们是来找你的。”“那你们……”“我们是来找你的，你们是不是在这里？”“我们是来找你的。”“那你们……”“我们是来找你的，我们的孩子，你们不要走，我们不要走，我们就不要走了。”“那你们怎么了？”“我们是不是走错路了？”“我们是不是走错路了？”“我们是不是有些不对劲了，你们不要走，我们就不能再走了。”“那么，你们不要走，我们不要走，我们就不会再来了。”“那怎么办？”“我们是不是走错路了？”“我们是不是走错路了？”“我们是不是有些不对劲，你们不要走，我们不要走，我们就不会再来了。”“那你们怎么走了？”“我们是不是走错路了？”“我们是不是有些不对劲，你们不是说好了吗？”“不，我们不是人，是鬼！”“我们是不是有些不对劲，你们不是说好了，你们不要走，我们不要走，我们就不会再来了。”“那你们怎么不去上班？”“我们是来找你的。”“那你们怎么不去上班？”“我们是来找你的。”“那你们怎么不去上班？”“我们是来找你的。”“那你们怎么不去上班？”“我们是来找你的。”“那你们怎么不去上班？”“我们是来找你的。”“那你们……”“我们是来找你的，你们是不是在这里？”“我们是来找你的。”“那你们……”“我们是来找你的。”“那你们怎么不去上班？”“我们是来找你的。”“那你们怎么不去上班？”“我们是来找你的。”“那你们怎么不去上班？”“我们是来找你的。”“那你说说，你们是不是在这里？”“我们是来找你的。'

In [23]:
len(result)

759

In [24]:
result.count("。")

20